### [Data Driven Attribution using Shapley value](https://towardsdatascience.com/data-driven-marketing-attribution-1a28d2e613a0)

In [1]:
channels = [
    'F',
    'O',
    'L'
]

coalitions = [
    'F',
    'O',
    'L',
    'F+O',
    'F+L',
    'L+O',
    'F+L+O'
]

conversion_ratio = [
    0.2,
    0.1,
    0.17,
    0.17,
    0.3,
    0.,
    0.06
]

In [2]:
from itertools import combinations, chain
data = list(
    chain(* [20*['+'.join(sorted(['F']))] + 
             10*['+'.join(sorted(['O']))] + 
             17*['+'.join(sorted(['L']))] + 
             17*['+'.join(sorted(['O','F']))] + 
             30*['+'.join(sorted(['L','F']))] + 
              0*['+'.join(sorted(['O','L']))] + 
              6*['+'.join(sorted(['F','O','L']))] ]
         )
)

data:

```sql
id|         path          |conversion
_____________________________________
 1|O(rganic)>F(acebook)   |    0
 2|O(rganic)              |    1
 3|L(inkedIn)             |    1
...
```

In [3]:
from random import sample 
sample(data, 10)

['F+L+O', 'F+O', 'F+L', 'L', 'L', 'F+O', 'L', 'F+O', 'F+L', 'F+L']

In [4]:
auto_coalitions = ['+'.join(sorted(x)) for x in list(chain(*[combinations(channels,l+1) for l in range(len(channels))]))]
assert coalitions == auto_coalitions

In [5]:
auto_conversion_ratio = [data.count(i)/len(data) for i in auto_coalitions]
assert conversion_ratio == auto_conversion_ratio

In [6]:
worth = []

for i, c1 in enumerate(auto_coalitions):
    w = auto_conversion_ratio[i]
    for j, c2 in enumerate(auto_coalitions):
        if len(c2) < len(c1) and len(list(set(c1.split('+')) & set(c2.split('+')))) > 0:
            w += auto_conversion_ratio[j]
    worth.append(w)

worth

[0.2, 0.1, 0.17, 0.47, 0.67, 0.27, 1.0]

In [7]:
from collections import defaultdict
from math import factorial
import copy

# COMPUTATION OF THE SHAPLEY VALUE

shapley = defaultdict(float)
n = len(channels)

for i in channels:
    for A in auto_coalitions:
        S = A.split('+')
        if i not in S:
            k = len(S) # Cardinality of set |S|
            Si = copy.deepcopy(S)
            Si.append(i)
            Si = '+'.join(sorted(Si))
            # Weight = |S|!(n-|S|-1)!/n!
            weight = (factorial(k) * factorial(n-k-1)) / factorial(n)
            # Marginal contribution = v(S U {i})-v(S)
            contrib = worth[auto_coalitions.index(Si)] - worth[auto_coalitions.index(A)]
            shapley[i] += weight * contrib
    shapley[i] += worth[auto_coalitions.index(i)]/n

shapley = { k: round(v, 3) for k, v in shapley.items() }

In [8]:
shapley

{'F': 0.455, 'O': 0.205, 'L': 0.34}